Nombre: Felix Gustavo Reyes

---


Implemtacion: Apache Pyspark

In [ ]:
!sudo apt install default-jre
!java -version

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
sc = pyspark.SparkContext(appName="yourAppName")
#reemplazarlo por el 7gigas
lista = [1,2,3,4,5]

In [ ]:
#creamos el rdd distribuido en RAM de los nodos
rdd= sc.parallelize(lista) 
rdd.collect()

[1, 2, 3, 4, 5]

In [ ]:
#SparkSQL y DataFrames
#MLib
#GraphX
#Spark Streming
#Transformaciones .map() .filter()
#Acciones: .count()  .first()

In [ ]:
#filtrado aqui poner drill
rdd_filtrado=rdd.filter(lambda x: x>1)
rdd_filtrado.collect()

[2, 3, 4, 5]

In [ ]:
#MAP
def suma(x):
  return(x+1)

In [ ]:
rdd_filtrado_sum=rdd_filtrado.map(suma)
rdd_filtrado_sum.collect()

[3, 4, 5, 6]

In [ ]:
rdd_cuadrado=(rdd_filtrado.map(suma).map(lambda x:(x,x**2)) )
rdd_cuadrado.collect()

[(3, 9), (4, 16), (5, 25), (6, 36)]

In [ ]:
#Flat Map
rdd_cuadrado=(rdd_filtrado.map(suma).flatMap(lambda x:(x,x**2)) )
rdd_cuadrado.collect()

[3, 9, 4, 16, 5, 25, 6, 36]

In [ ]:
#Muestras
rdd_cuadrado.sample(False,1).collect()

[3, 9, 4, 16, 5, 25, 6, 36]

In [ ]:
rdd_cuadrado.sample(False,0.5).collect()

[3, 9, 4, 5, 25, 6, 36]

In [ ]:
rdd_cuadrado.sample(True,1).collect()

[3, 9, 4, 16, 5, 5, 5, 25, 6, 36]

In [ ]:
#Quitar Duplicados
lista=[1,1,2,2,3,3,4,4]
rdd=sc.parallelize(lista)
rdd.collect()

[1, 1, 2, 2, 3, 3, 4, 4]

In [ ]:
rdd.distinct().collect()

[2, 4, 1, 3]

In [ ]:
#Agrupado de Datos en formato clave valor
rdd_agrupado=rdd.groupBy(lambda x: x>1)
rdd_agrupado.collect()

[(False, <pyspark.resultiterable.ResultIterable at 0x7f7ebf4447b8>),
 (True, <pyspark.resultiterable.ResultIterable at 0x7f7ebf444828>)]

In [ ]:
print([(x,sorted(y)) for (x,y) in rdd_agrupado.collect()])

[(False, [1, 1]), (True, [2, 2, 3, 3, 4, 4])]


In [ ]:
#Transformacion en varios RDD
#Union
rdda=sc.parallelize([1,2,3])
rddb=sc.parallelize([4,5,6])
rddu=rdda.union(rddb)
rddu.collect()

[1, 2, 3, 4, 5, 6]

In [ ]:
#Interseccion
rdda=sc.parallelize([1,2,3])
rddb=sc.parallelize([3,5,6])
rddi=rdda.intersection(rddb)
rddi.collect()

[3]

In [ ]:
#Substraccion
rdds=rdda.subtract(rddb)
rdds.collect()

[1, 2]

In [ ]:
#Producto Cartesiano
rddc=rdda.cartesian(rddb)
rddc.collect()

[(1, 3), (1, 5), (1, 6), (2, 3), (3, 3), (2, 5), (2, 6), (3, 5), (3, 6)]

In [ ]:
spark = SparkSession.builder.appName("ParquetInputFormat").getOrCreate()
path = /user/root/ROLES""
#sc = spark.sparkContext()
parquet_rdd = sc.newAPIHadoopFile(
    path,
    'org.apache.parquet.avro.AvroParquetInputFormat',
    'java.lang.Void',
    'org.apache.avro.generic.IndexedRecord',
    valueConverter='org.apache.spark.examples.pythonconverters.IndexedRecordToJavaConverter')
output = parquet_rdd.map(lambda x: x[1]).collect()
for k in output:
    print(k)



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
import pyspark
from __future__ import print_function
from pyspark.sql import SparkSession
# $example on:schema_merging$
from pyspark.sql import Row
# $example off:schema_merging$

In [ ]:
import os
path="/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/"
os.chdir(path)
!ls

employees.json	kv1.txt     people.json  user.avsc   users.orc
full_user.avsc	people.csv  people.txt	 users.avro  users.parquet


###basic_datasource_example

In [ ]:
def basic_datasource_example(spark):
    # $example on:generic_load_save_functions$
    df = spark.read.load("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/users.parquet")
    df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")
    # $example off:generic_load_save_functions$

    # $example on:write_partitioning$
    df.write.partitionBy("favorite_color").format("parquet").save("namesPartByColor.parquet")
    # $example off:write_partitioning$

    # $example on:write_partition_and_bucket$
    df = spark.read.parquet("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/users.parquet")
    (df
        .write
        .partitionBy("favorite_color")
        .bucketBy(42, "name")
        .saveAsTable("people_partitioned_bucketed"))
    # $example off:write_partition_and_bucket$

    # $example on:manual_load_options$
    df = spark.read.load("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.json", format="json")
    df.select("name", "age").write.save("namesAndAges.parquet", format="parquet")
    # $example off:manual_load_options$

    # $example on:manual_load_options_csv$
    df = spark.read.load("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.csv",
                         format="csv", sep=":", inferSchema="true", header="true")
    # $example off:manual_load_options_csv$

    # $example on:manual_save_options_orc$
    df = spark.read.orc("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/users.orc")
    (df.write.format("orc")
        .option("orc.bloom.filter.columns", "favorite_color")
        .option("orc.dictionary.key.threshold", "1.0")
        .save("users_with_options.orc"))
    # $example off:manual_save_options_orc$

    # $example on:write_sorting_and_bucketing$ 
    #name, favorite_color, favorite_numbers
    #df.write.bucketBy(42, "name").sortBy("age").saveAsTable("people_bucketed")
    df.write.bucketBy(42, "name").sortBy("favorite_color").saveAsTable("people_bucketed")    
    # $example off:write_sorting_and_bucketing$

    # $example on:direct_sql$
    df = spark.sql("SELECT * FROM parquet.`/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/users.parquet`")
    # $example off:direct_sql$

    spark.sql("DROP TABLE IF EXISTS people_bucketed")
    spark.sql("DROP TABLE IF EXISTS people_partitioned_bucketed")


###parquet_example

In [ ]:
def parquet_example(spark):
    # $example on:basic_parquet_example$
    peopleDF = spark.read.json("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.json")

    # DataFrames can be saved as Parquet files, maintaining the schema information.
    peopleDF.write.parquet("people.parquet")

    # Read in the Parquet file created above.
    # Parquet files are self-describing so the schema is preserved.
    # The result of loading a parquet file is also a DataFrame.
    parquetFile = spark.read.parquet("/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.parquet")

    # Parquet files can also be used to create a temporary view and then used in SQL statements.
    parquetFile.createOrReplaceTempView("parquetFile")
    teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
    teenagers.show()
    # +------+
    # |  name|
    # +------+
    # |Justin|
    # +------+
    # $example off:basic_parquet_example$



###parquet_schema_merging_example

In [ ]:
def parquet_schema_merging_example(spark):
    # $example on:schema_merging$
    # spark is from the previous example.
    # Create a simple DataFrame, stored into a partition directory
    sc = spark.sparkContext

    squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                      .map(lambda i: Row(single=i, double=i ** 2)))
    squaresDF.write.parquet("data/test_table/key=1")

    # Create another DataFrame in a new partition directory,
    # adding a new column and dropping an existing column
    cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                    .map(lambda i: Row(single=i, triple=i ** 3)))
    cubesDF.write.parquet("data/test_table/key=2")

    # Read the partitioned table
    mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
    mergedDF.printSchema()

    # The final schema consists of all 3 columns in the Parquet files together
    # with the partitioning column appeared in the partition directory paths.
    # root
    #  |-- double: long (nullable = true)
    #  |-- single: long (nullable = true)
    #  |-- triple: long (nullable = true)
    #  |-- key: integer (nullable = true)
    # $example off:schema_merging$




###json_dataset_example

In [ ]:
def json_dataset_example(spark):
    # $example on:json_dataset$
    # spark is from the previous example.
    sc = spark.sparkContext

    # A JSON dataset is pointed to by path.
    # The path can be either a single text file or a directory storing text files
    path = "/content/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.json"
    peopleDF = spark.read.json(path)

    # The inferred schema can be visualized using the printSchema() method
    peopleDF.printSchema()
    # root
    #  |-- age: long (nullable = true)
    #  |-- name: string (nullable = true)

    # Creates a temporary view using the DataFrame
    peopleDF.createOrReplaceTempView("people")

    # SQL statements can be run by using the sql methods provided by spark
    teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
    teenagerNamesDF.show()
    # +------+
    # |  name|
    # +------+
    # |Justin|
    # +------+

    # Alternatively, a DataFrame can be created for a JSON dataset represented by
    # an RDD[String] storing one JSON object per string
    jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']
    otherPeopleRDD = sc.parallelize(jsonStrings)
    otherPeople = spark.read.json(otherPeopleRDD)
    otherPeople.show()
    # +---------------+----+
    # |        address|name|
    # +---------------+----+
    # |[Columbus,Ohio]| Yin|
    # +---------------+----+
    # $example off:json_dataset$


###jdbc_dataset_example

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.6.jar

--2020-12-13 04:35:18--  https://jdbc.postgresql.org/download/postgresql-42.2.6.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 842825 (823K) [application/java-archive]
Saving to: ‘postgresql-42.2.6.jar’

postgresql-42.2.6.j 100%[===================>] 823.07K  --.-KB/s    in 0.1s    

2020-12-13 04:35:18 (7.86 MB/s) - ‘postgresql-42.2.6.jar’ saved [842825/842825]



In [ ]:
def jdbc_dataset_example(spark):
    # $example on:jdbc_dataset$
    # Note: JDBC loading and saving can be achieved via either the load/save or jdbc methods
    # Loading data from a JDBC source    
    #.option("query", "SELECT *  FROM tblDatos")
    jdbcDF = spark.read \
        .format("jdbc") \
        .option("database", "") \
        .option("dbtable", "") \
        .option("url", "") \
        .option("user", "") \
        .option("password", "") \
        .load()

        #.option("encrypt", "true") \
        #.option("trustServerCertificate", "false") \
        #.option("hostNameInCertificate", "*.database.windows.net") \
        #.option("loginTimeout", "30") \

    #jdbcDF2 = spark.read \
    #    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
    #          properties={"user": "username", "password": "password"})

    # Specifying dataframe column data types on read
    #jdbcDF3 = spark.read \
    #    .format("jdbc") \
    #    .option("url", "jdbc:postgresql:dbserver") \
    #    .option("dbtable", "schema.tablename") \
    #    .option("user", "username") \
    #    .option("password", "password") \
    #    .option("customSchema", "id DECIMAL(38, 0), name STRING") \
    #    .load()

    # Saving data to a JDBC source
    #jdbcDF.write \
    #    .format("jdbc") \
    #    .option("url", "jdbc:postgresql:dbserver") \
    #    .option("dbtable", "schema.tablename") \
    #    .option("user", "username") \
    #    .option("password", "password") \
    #    .save()

    #jdbcDF2.write \
    #    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
    #          properties={"user": "username", "password": "password"})

    # Specifying create table column data types on write
    #jdbcDF.write \
    #    .option("createTableColumnTypes", "name CHAR(64), comments VARCHAR(1024)") \
    #    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
    #          properties={"user": "username", "password": "password"})
    # $example off:jdbc_dataset$

###Main

In [ ]:
    spark = SparkSession.builder.appName("Python Spark SQL data source example").getOrCreate()
    basic_datasource_example(spark)
    parquet_example(spark)
    parquet_schema_merging_example(spark)
    json_dataset_example(spark)
    #jdbc_dataset_example(spark)
    spark.stop()